In [1]:
import os
import s3fs
import pandas as pd
import functions as fc 

In [2]:
df_rep_pm = fc.import_inpi( "societe_rep.csv", cols=None)

/home/coder/work/data_inpi/code/functions.py:192: DtypeWarning: Columns (14,16,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8",


In [3]:
df_rep_pm.head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,qualite_cor,denomination_cor,commune,code_departement,code_commune_insee,code_commune_cor,adresse_ligne1_cor,adresse_ligne2_cor,adresse_ligne3_cor,qualite_simplifie
0,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,...,président,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0,84031.0,1360 chemin saint-roch,1360 chemin saint-roch,NaN,1.0
1,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,directeur général,societe immobiliere financiere d etudes et de ...,paris,75,NaN,NaN,50 rue de la tour,50 rue de la tour,NaN,1.0
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,...,directeur général,societe immobiliere financiere d etudes et de ...,carpentras,84,84031.0,84031.0,1360 chemin saint-roch,1360 chemin saint-roch,NaN,1.0
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,VATIN-CHAIX,Bertrand Heinrich René,P.Physique,1977-06-09,...,directeur général,societe immobiliere financiere d etudes et de ...,strasbourg,67,67482.0,67482.0,63 rue virgile,63 rue virgile,NaN,1.0
4,101,61500740,SOCIETE J.F.B.,Société par actions simplifiée,Président,BENOIT,NaN,Nicolas Arnold Denis,P.Physique,1974-05-15,...,président,societe j.f.b.,ars-sur-formans,14,NaN,1021.0,222 rue jean-marie vianney,222 rue jean-marie vianney,NaN,1.0


In [3]:
deces = fc.import_file("deces/deces_all_years.csv")

/home/coder/work/data_inpi/code/functions.py:269: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_rep_pm = pd.read_csv(file_in, sep=";", encoding="utf-8")


In [4]:
deces.head()

,sexe,datenaiss,lieunaiss,commnaiss,paysnaiss,datedeces,lieudeces,nom,prenom,annee
0,2,19221022,01366,SAINTE-JULIE,NaN,20160114,01004,MARIN,MARCELINE,2016
1,2,19311101,62617,NOEUX-LES-MINES,NaN,20160105,01004,DEBRET,THERESE MADELEINE,2016
2,1,19410508,42147,MONTBRISON,NaN,20160121,01004,THELIOL,ANTOINE ROGER MARIE,2016
3,1,19320207,01004,AMBERIEU-EN-BUGEY,NaN,20160121,01004,GALLET,ROBERT HENRI,2016
4,2,19300218,01004,AMBERIEU-EN-BUGEY,NaN,20160122,01004,BOUCHARD,GEORGETTE,2016


In [6]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'nom_patronymique_cor', 'qualite_cor', 'denomination_cor', 'commune',
       'code_departement', 'code_commune_insee', 'code_commune_cor',
       'adresse_ligne1_cor', 'adresse_ligne2_cor', 'adresse_ligne3_cor',
       'qualite_simplifie'],
      dtype='object')

In [45]:
# variables permettant de lier le fichier deces au fichier inpi:
df_rep_pm.loc[:,["nom_patronymique_cor", "prenoms", "nom_usage", "date_naissance", 'ville_naissance']].head()

,nom_patronymique_cor,prenoms,nom_usage,date_naissance,ville_naissance
0,VATIN,Gerard,NaN,1947-10-07,79270 Saint-Maixent-L'École 79
1,VATIN,Jacques,NaN,1930-04-03,02691 Saint-Quentin 02
2,JORDENS,Regina,VATIN,1948-08-07,Allemagne
3,VATIN,Bertrand Heinrich René,VATIN-CHAIX,1977-06-09,Chêne bougeries
4,BENOIT,Nicolas Arnold Denis,NaN,1974-05-15,69381 LYON 01


In [27]:
# il faut nettoyer la ville de naissance dans la base inpi et y ajouter le code commune insee

df_rep_pm["commune_nais"] = df_rep_pm["ville_naissance"].str.lower()

In [28]:
df_rep_pm["commune_nais"] = (df_rep_pm["commune_nais"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [29]:
re0 = "([0-9]{5})"
df_rep_pm["code_commune_nais"]  = df_rep_pm["commune_nais"].str.extract(re0)

In [30]:
df_rep_pm.loc[:,["code_commune_nais", "commune_nais"]].head(10)

,code_commune_nais,commune_nais
0,79270,79270 saint-maixent-l'ecole 79
1,02691,02691 saint-quentin 02
2,NaN,allemagne
3,NaN,chêne bougeries
4,69381,69381 lyon 01
5,38185,38185 grenoble
6,69386,69386 lyon 6eme
7,69386,69386 lyon 6eme
8,35238,35238 rennes
9,35335,35335 thourie


In [31]:
df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re0, "")

/tmp/ipykernel_3684/1642337416.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re0, "")


In [32]:
df_rep_pm.loc[:,["code_commune_nais", "commune_nais"]].head(10)

,code_commune_nais,commune_nais
0,79270,saint-maixent-l'ecole 79
1,02691,saint-quentin 02
2,NaN,allemagne
3,NaN,chêne bougeries
4,69381,lyon 01
5,38185,grenoble
6,69386,lyon 6eme
7,69386,lyon 6eme
8,35238,rennes
9,35335,thourie


In [33]:
re1 = "[0-9]{1,5}"
df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re1, "")

/tmp/ipykernel_3684/672803900.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re1, "")


In [34]:
df_rep_pm.loc[:,["code_commune_nais", "commune_nais"]].head(10)

,code_commune_nais,commune_nais
0,79270,saint-maixent-l'ecole
1,02691,saint-quentin
2,NaN,allemagne
3,NaN,chêne bougeries
4,69381,lyon
5,38185,grenoble
6,69386,lyon eme
7,69386,lyon eme
8,35238,rennes
9,35335,thourie


In [35]:
re2 = "\seme"
df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re2, "")

/tmp/ipykernel_3684/1312444802.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re2, "")


In [36]:
df_rep_pm.loc[:,["code_commune_nais", "commune_nais"]].head(10)

,code_commune_nais,commune_nais
0,79270,saint-maixent-l'ecole
1,02691,saint-quentin
2,NaN,allemagne
3,NaN,chêne bougeries
4,69381,lyon
5,38185,grenoble
6,69386,lyon
7,69386,lyon
8,35238,rennes
9,35335,thourie


In [38]:
df_rep_pm["commune_nais"].value_counts()

 paris                   183100
 paris ()                178503
 lyon ()                 122217
 paris                    88267
 marseille ()             78405
                          ...  
 geneve (suisse               1
 ochey (mm)                   1
 frohmuhl (bas rhin)          1
 haussonville (mm)            1
 besancon - doubs             1
Name: commune_nais, Length: 307313, dtype: int64

In [39]:
re3 = "\(\)"
df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re3, "")

/tmp/ipykernel_3684/2145190911.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep_pm["commune_nais"]  = df_rep_pm["commune_nais"].str.replace(re3, "")


In [40]:
df_rep_pm["commune_nais"].value_counts()

 paris                           362005
 lyon                            135556
 marseille                       128079
 paris                            96312
 toulouse                         86182
                                  ...  
 airaes felguerias                    1
 vieira-de-leira                      1
 fresnes sur apance                   1
 onondaga (etat de new york)          1
 besancon - doubs                     1
Name: commune_nais, Length: 274752, dtype: int64

In [41]:
df_rep_pm["commune_nais"] = df_rep_pm["commune_nais"].str.strip()

In [42]:
df_rep_pm["commune_nais"].value_counts()

paris               468617
lyon                157761
marseille           140425
toulouse             88219
nantes               67349
                     ...  
yukarikale               1
ortavoy                  1
findiksuyu               1
dezfool                  1
besancon - doubs         1
Name: commune_nais, Length: 238275, dtype: int64

In [43]:
df_rep_pm.loc[df_rep_pm["code_commune_nais"].isna()==False, :].shape # 5 501 010, on ne sait pas si c'est code commune ou code postal

(5501010, 32)

In [46]:
deces["datenaiss"].head()

0    19221022
1    19311101
2    19410508
3    19320207
4    19300218
Name: datenaiss, dtype: int64

In [51]:
deces["date_naissance"] = deces["datenaiss"].astype(str)

In [54]:
deces["date_naissance"].str.slice(0, 4).head()

0    1922
1    1931
2    1941
3    1932
4    1930
Name: date_naissance, dtype: object

In [55]:
deces["date_naissance"] = deces["date_naissance"].str.slice(0, 4) + "-" + deces["date_naissance"].str.slice(4,6) + "-" + deces["date_naissance"].str.slice(6,8)

In [56]:
deces["date_naissance"].head()

0    1922-10-22
1    1931-11-01
2    1941-05-08
3    1932-02-07
4    1930-02-18
Name: date_naissance, dtype: object

In [57]:
deces["nom_cor"] = deces["nom"].str.lower()

In [58]:
deces["nom_cor"].head()

0       marin
1      debret
2     theliol
3      gallet
4    bouchard
Name: nom_cor, dtype: object

In [59]:
df_rep_pm["nom_patronymique_cor"] = df_rep_pm["nom_patronymique_cor"].str.lower()

In [60]:
df_rep_pm.loc[df_rep_pm["nom_usage"].isna() == False,:].shape # 1 350 973

(1350973, 32)

In [61]:
df_rep_pm.loc[(df_rep_pm["nom_patronymique_cor"].isna())&(df_rep_pm["nom_usage"].isna() == False),:].shape  # 7

(7, 32)

In [62]:
df_rep_pm["nom_usage_cor"] = df_rep_pm["nom_usage"].str.lower()

In [63]:
df_rep_pm["nom_usage_cor"].head()

0            NaN
1            NaN
2          vatin
3    vatin-chaix
4            NaN
Name: nom_usage_cor, dtype: object

In [ ]:
# on va faire la jointure entre nom d'usage et nom dans le fichier décès, sauf si nom d'usage n'est pas rempli et dans ce cas on utilisera nom patronymique

In [72]:
# prénoms
deces["prenom"].head()

0              marceline
1      therese madeleine
2    antoine roger marie
3           robert henri
4              georgette
Name: prenom, dtype: object

In [71]:
deces["prenom"] = deces["prenom"].str.lower()
deces["prenom"] = deces["prenom"].str.strip()

In [66]:
df_rep_pm["prenoms"].head()

0                    Gerard
1                   Jacques
2                    Regina
3    Bertrand Heinrich René
4      Nicolas Arnold Denis
Name: prenoms, dtype: object

In [67]:
df_rep_pm["prenoms_cor"] = df_rep_pm["prenoms"].str.lower()

In [68]:
df_rep_pm["prenoms_cor"] = (df_rep_pm["prenoms_cor"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))

In [73]:
df_rep_pm["prenoms_cor"] = df_rep_pm["prenoms_cor"].str.strip()

In [74]:
df_rep_pm["prenoms_cor"].head()

0                    gerard
1                   jacques
2                    regina
3    bertrand heinrich rene
4      nicolas arnold denis
Name: prenoms_cor, dtype: object

In [75]:
deces.columns

Index(['sexe', 'datenaiss', 'lieunaiss', 'commnaiss', 'paysnaiss', 'datedeces',
       'lieudeces', 'nom', 'prenom', 'annee', 'date_naissance', 'nom_cor'],
      dtype='object')

In [83]:
test = deces.duplicated(subset=["prenom", "nom_cor", "date_naissance", "commnaiss"])

In [84]:
test.sum() # 815

0

In [81]:
deces = deces.drop_duplicates(subset=["prenom", "nom_cor", "date_naissance", "commnaiss"])

In [82]:
deces.shape # 3812318

(3812318, 12)

In [86]:
df_rep_pm.columns

Index(['code_greffe', 'siren', 'denomination', 'forme_juridique', 'qualite',
       'nom_patronymique', 'nom_usage', 'prenoms', 'type', 'date_naissance',
       'ville_naissance', 'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3',
       'code_postal', 'ville', 'code_commune', 'pays', 'id_representant',
       'nom_patronymique_cor', 'qualite_cor', 'denomination_cor', 'commune',
       'code_departement', 'code_commune_insee', 'code_commune_cor',
       'adresse_ligne1_cor', 'adresse_ligne2_cor', 'adresse_ligne3_cor',
       'qualite_simplifie', 'commune_nais', 'code_commune_nais',
       'nom_usage_cor', 'prenoms_cor'],
      dtype='object')

In [87]:
deces = deces.rename({"lieunaiss" : "code_commune_nais", "nom_cor" : "nom_usage_cor", "prenom": "prenoms_cor"}, axis=1)

In [92]:
df_rep_pm.loc[(df_rep_pm["nom_patronymique_cor"].isna() == False) & (df_rep_pm["nom_usage_cor"].isna()), 'nom_usage_cor'] = \
    df_rep_pm.loc[(df_rep_pm["nom_patronymique_cor"].isna() == False) & (df_rep_pm["nom_usage_cor"].isna()), 'nom_patronymique_cor'] 

In [102]:
deces["commnaiss"].head()

0         SAINTE-JULIE
1      NOEUX-LES-MINES
2           MONTBRISON
3    AMBERIEU-EN-BUGEY
4    AMBERIEU-EN-BUGEY
Name: commnaiss, dtype: object

In [103]:
deces["commune_nais"] = deces["commnaiss"].str.lower()

In [104]:
deces["commune_nais"].value_counts()

paris 14e  arrondissement    46888
marseille                    44224
departement d'oran           23622
paris 15e  arrondissement    21915
paris 10e  arrondissement    21203
                             ...  
magasfok                         1
passos s juliao - braga          1
coronado santo tirso             1
campia turzii, cluj              1
issaiden                         1
Name: commune_nais, Length: 157119, dtype: int64

In [105]:
deces["commune_nais"].isna().sum() # 3504

3504

In [106]:
re0 = "([0-9]{1,2}\s+arrondissement|[0-9]{1,2}e\s+arrondissement|[0-9]{1,2}eme\s+arrondissement)"
deces["commune_nais"] = deces["commune_nais"].str.replace(re0, "")

/tmp/ipykernel_3684/55226059.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  deces["commune_nais"] = deces["commune_nais"].str.replace(re0, "")


In [107]:
deces["commune_nais"].value_counts()

paris                 210786
marseille              44224
lyon                   42249
departement d'oran     23622
lille                  20898
                       ...  
east challow               1
nova lima (mines)          1
moerbeke-waas              1
cagnon                     1
issaiden                   1
Name: commune_nais, Length: 157064, dtype: int64

In [108]:
df_rep_pm["commune_nais"].value_counts()

paris               468617
lyon                157761
marseille           140425
toulouse             88219
nantes               67349
                     ...  
yukarikale               1
ortavoy                  1
findiksuyu               1
dezfool                  1
besancon - doubs         1
Name: commune_nais, Length: 238275, dtype: int64

In [93]:
df_rep_pm["nom_patronymique_cor"].head()

0      vatin
1      vatin
2    jordens
3      vatin
4     benoit
Name: nom_patronymique_cor, dtype: object

In [110]:
deces.columns

Index(['sexe', 'datenaiss', 'code_commune_nais', 'commnaiss', 'paysnaiss',
       'datedeces', 'lieudeces', 'nom', 'prenoms_cor', 'annee',
       'date_naissance', 'nom_usage_cor', 'commune_nais'],
      dtype='object')

In [111]:
test = deces.duplicated(subset=["prenoms_cor", "nom_usage_cor", "date_naissance", "commune_nais"])

In [112]:
test.sum()

0

In [113]:
test = pd.merge(df_rep_pm, deces, on=["date_naissance", "commune_nais", "nom_usage_cor", "prenoms_cor"], how="left")

In [114]:
test.shape #7384566

(7384566, 43)

In [96]:
df_rep_pm.shape

(7384566, 34)

In [115]:
test.head()

,code_greffe,siren,denomination,forme_juridique,qualite,nom_patronymique,nom_usage,prenoms,type,date_naissance,...,prenoms_cor,sexe,datenaiss,code_commune_nais_y,commnaiss,paysnaiss,datedeces,lieudeces,nom,annee
0,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Président,VATIN,NaN,Gerard,P.Physique,1947-10-07,...,gerard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,NaN,Jacques,P.Physique,1930-04-03,...,jacques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,JORDENS,VATIN,Regina,P.Physique,1948-08-07,...,regina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101,26380212,SOCIETE IMMOBILIERE FINANCIERE D ETUDES ET DE ...,Société par actions simplifiée,Directeur général,VATIN,VATIN-CHAIX,Bertrand Heinrich René,P.Physique,1977-06-09,...,bertrand heinrich rene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101,61500740,SOCIETE J.F.B.,Société par actions simplifiée,Président,BENOIT,NaN,Nicolas Arnold Denis,P.Physique,1974-05-15,...,nicolas arnold denis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
test["datedeces"].isna().sum()  # 7384472

7384472

In [116]:
test.loc[test["datedeces"].isna() == False,:].head()

Error: Canceled future for execute_request message before replies were done

In [100]:
test.loc[test["datedeces"].isna() == False,:].shape # 94

(94, 42)